# Кибериммунная автономность$\\$Создание конструктивно защищённого автономного наземного транспортного средства$\\$Модуль 2

## О документе

Версия 1.03

Модуль 2 для регионального этапа соревнований по кибериммунной автономности

## Подготовка к работе c модулем 2

Перенесите в блок ниже ваши реализации классов коммуникационного шлюза (CommunicationGateway), систему навигации (NavigationSystem), управления (ControlSystem)

In [286]:
from queue import Queue
from src.communication_gateway import BaseCommunicationGateway
from src.config import CARGO_BAY_QUEUE_NAME, CONTROL_SYSTEM_QUEUE_NAME
from src.event_types import Event
from src.navigation_system import BaseNavigationSystem
from src.config import SERVOS_QUEUE_NAME
from src.control_system import BaseControlSystem


class NavigationSystem(BaseNavigationSystem):
    """ класс навигационного блока """
    def _send_position_to_consumers(self):
        # Передаём в очередь Системы управления        
        control_q_name = CONTROL_SYSTEM_QUEUE_NAME
        
        # Формируем объект для передачи координат 
        event = Event(source=self.event_source_name,
                      destination=control_q_name,
                      operation='position_update',
                      parameters=self._position) 
        
        # помещаем в очередь
        control_q: Queue = self._queues_dir.get_queue(control_q_name)
        control_q.put(event)


class CommunicationGateway(BaseCommunicationGateway):
    """CommunicationGateway блок связи"""
    def _send_mission_to_consumers(self):
        # отправка сегмента маршрута в систему управления
        event = Event(source=BaseCommunicationGateway.event_source_name,
                        destination=CONTROL_SYSTEM_QUEUE_NAME,
                        operation="set_mission",
                        parameters=self._mission
        )

        control_q: Queue = self._queues_dir.get_queue(CONTROL_SYSTEM_QUEUE_NAME)
        # помещаем в очередь
        control_q.put(event)


class ControlSystem(BaseControlSystem):
    """ControlSystem блок расчёта управления """
    def _send_speed_and_direction_to_consumers(self, speed, direction):
        # Очередь управления модулями
        servos_q_name = SERVOS_QUEUE_NAME 
        servos_q: Queue = self._queues_dir.get_queue(servos_q_name)

        # отправка сообщения с указанной скоростью
        event_speed = Event(source=self.event_source_name,
                            destination=servos_q_name,
                            operation='set_speed',
                            parameters=speed)

        # отправка сообщения с указанным направлением
        event_direction = Event(source=self.event_source_name,
                                destination=servos_q_name,
                                operation='set_direction',
                                parameters=direction) 
        # помещаем в очередь
        servos_q.put(event_speed)
        servos_q.put(event_direction) 



    def _lock_cargo(self):
        """ заблокировать грузовой отсек """
        # получаем очередь грузового отсека из queues_dir
        cargo_q = self._queues_dir.get_queue(CARGO_BAY_QUEUE_NAME)

        # формируем объект с командой "закрыть грузовой отсек"
        event = Event(source=CONTROL_SYSTEM_QUEUE_NAME,
                      destination=CARGO_BAY_QUEUE_NAME,
                      operation='lock_cargo',
                      parameters=None)
        # помещаем в очередь
        cargo_q.put(event)

    def _release_cargo(self):
        """ открыть грузовой отсек """
        # получаем очередь грузового отсека из queues_dir
        cargo_q = self._queues_dir.get_queue(CARGO_BAY_QUEUE_NAME)

        # формируем объект с командой "закрыть грузовой отсек"
        event = Event(source=CONTROL_SYSTEM_QUEUE_NAME,
                      destination=CARGO_BAY_QUEUE_NAME,
                      operation='release_cargo',
                      parameters=None)

        # помещаем в очередь
        cargo_q.put(event)



Если у вас настроена и работает СУПА, установите в True значение переменной afcs_present

In [287]:
afcs_present = True

В этом модуле изменим идентификатор машинки:

In [288]:
car_id = "m2" 

### Модуль 2. Отправка маршрутного здания и следование по маршруту

В рамках этого модуля отрабатывается только автономность. 
Никаких киберпрепятствий пока не будет. 

Что нужно сделать по пунктам:
1. Для заданных начальных и конечных точек создать маршрут, содержащий не менее пяти промежуточных путевых точек вручную или с использованием таких инструментов как APMPlanner, MissionPlanner. <br>Содержимое wpl файла нужно вставить в соответствующий кодовый блок блокнота, запустить симуляцию и убедиться, что машинка успешно проходит этот маршрут.
2. Задать скоростные ограничения в виде массива элементов GeoSpecificSpeedLimit
3. Запустить симуляцию


Формат WPL файла описан на этой странице: 

https://mavlink.io/en/file_formats/#mission_plain_text_file

Пример содержимого файла с текстовым описанием маршрута

In [289]:
wpl_file_content =  """QGC WPL 110
0	1	0	16	0	5	0	0	55.75995473043264   49.19218064371151	0	1
1	0	3	16	0	5	0	0	55.76108655554123   49.19003664121742	0	1
2	0	3	16	0	5	0	0	55.76546198801143   49.18248805736174	0	1
3	0	3	16	0	5	0	0	55.766007359326046  49.181489406029     0  	1
4	0	3	16	0	5	0	0	55.766118346337585  49.181495192435484  0	1

5	0	3	16	0	5	0	0	55.768766230077766  49.18822890829518   0	1
6	0	3	16	0	5	0	0	55.76887707561642   49.18825664346313	0	1
7	0	3	16	0	5	0	0	55.7689724999394    49.188202250817156  0	1

8	0	3	16	0	5	0	0	55.77117201592831   49.185591787353246  0	1
9	0	3	16	0	5	0	0	55.771281087668925  49.18550136497075   0	1
10	0	3	16	0	5	0	0	55.77135568578326   49.18549433211953   0	1
11	0	3	16	0	5	0	0	55.77189361893662   49.18643675349769   0	1

12	0	3	16	0	5	0	0	55.77222781145287  49.18750515264204    0	1
13	0	3	16	0	5	0	0	55.77224637570111  49.18762256067702    0	1

14	0	3	16	0	5	0	0	55.77220925536136  49.18856509489694    0	1

15	0	3	16	0	5	0	0	55.77293427665861  49.192796833521165   0	1
16	0	3	16	0	5	0	0	55.773368983625446 49.19649189232489    0	1

17	0	3	16	0	5	0	0	55.77361869823526  49.19813877193922    0	1
18	0	3	16	0	5	0	0	55.773535467090205 49.198396470354034   0	1


19	0	3	16	0	5	0	0	55.76519548049886   49.20245953776684   0	1
20  0   3   16  0   5   0   0   55.76503745948515   49.20266237632634   0   1
"""


Для выполнения задания в блоке ниже  
1. замените имя файла с маршрутом (на свой), вот эту строку:
   ```python
   wpl_file = "module2.wpl"
   ```
2. измените список скоростных ограничений, вот этот массив:
   ```python
   speed_limits = [
    GeoSpecificSpeedLimit(...
    ```
   
Учтите, что вам нужно доставить груз как можно быстрее, но при этом не нарушить правила дорожного движения. 

Ограничения скорости
- населённый пункт - 60 км/ч
- пешеходная зона - 20 км/ч
- магистраль - 110 км/ч

Составьте маршрут доставки так, чтобы машинка доехала за минимальное время. Для определения оптимального маршрута допустимо пользоваться сторонними сервисами (например, Яндекс картами).

In [290]:
# создадим wpl файл и запишем туда маршрутное задание
wpl_file = "module2.wpl"

**Примечание**:
После создания собственного маршрута удалите перезапись содержимого, чтобы не потерять свои наработки!


In [291]:
with open(wpl_file, "w") as f:
    f.write(wpl_file_content)

**Напоминание**: не забудьте изменить скоростные ограничения (speed_limits) при задании маршрутного задания!

In [292]:
from src.wpl_parser import WPLParser

parser = WPLParser(wpl_file)    
points = parser.parse()
print(points)

from src.mission_type import GeoSpecificSpeedLimit
speed_limits = [
    GeoSpecificSpeedLimit(0, 80),
    GeoSpecificSpeedLimit(2, 45),
    GeoSpecificSpeedLimit(3, 30),
    GeoSpecificSpeedLimit(4, 60),

    GeoSpecificSpeedLimit(5, 30),
    GeoSpecificSpeedLimit(7, 80),

    GeoSpecificSpeedLimit(9, 30),
    GeoSpecificSpeedLimit(11, 60),

    GeoSpecificSpeedLimit(14, 80),
    GeoSpecificSpeedLimit(16, 45),

    GeoSpecificSpeedLimit(17, 30),
    GeoSpecificSpeedLimit(18, 90),

    GeoSpecificSpeedLimit(19, 20),
]

from src.mission_planner import Mission

home = points[0]
mission = Mission(home=home, waypoints=points,speed_limits=speed_limits, armed=True)

[Point(55.75995473043264, 49.19218064371151, 0.0), Point(55.76108655554123, 49.19003664121742, 0.0), Point(55.76546198801143, 49.18248805736174, 0.0), Point(55.766007359326046, 49.181489406029, 0.0), Point(55.766118346337585, 49.181495192435484, 0.0), Point(55.768766230077766, 49.18822890829518, 0.0), Point(55.76887707561642, 49.18825664346313, 0.0), Point(55.7689724999394, 49.188202250817156, 0.0), Point(55.77117201592831, 49.185591787353246, 0.0), Point(55.771281087668925, 49.18550136497075, 0.0), Point(55.77135568578326, 49.18549433211953, 0.0), Point(55.77189361893662, 49.18643675349769, 0.0), Point(55.77222781145287, 49.18750515264204, 0.0), Point(55.77224637570111, 49.18762256067702, 0.0), Point(55.77220925536136, 49.18856509489694, 0.0), Point(55.77293427665861, 49.192796833521165, 0.0), Point(55.773368983625446, 49.19649189232489, 0.0), Point(55.77361869823526, 49.19813877193922, 0.0), Point(55.773535467090205, 49.198396470354034, 0.0), Point(55.76519548049886, 49.2024595377668

Запускаем симуляцию с новым маршрутным заданием

In [293]:
from time import sleep
from geopy import Point as GeoPoint


from src.queues_dir import QueuesDirectory
from src.servos import Servos
from src.sitl import SITL
from src.cargo_bay import CargoBay
from src.mission_planner import MissionPlanner
from src.config import LOG_ERROR, LOG_INFO
from src.mission_planner_mqtt import MissionSender
from src.sitl_mqtt import TelemetrySender
from src.system_wrapper import SystemComponentsContainer


# координата текущего положения машинки
home = GeoPoint(latitude=55.759958, longitude=49.192199) 


# каталог очередей для передачи сообщений между блоками
queues_dir = QueuesDirectory() 

if afcs_present:
    mission_sender = MissionSender(
        queues_dir=queues_dir, client_id=car_id, log_level=LOG_ERROR)
    telemetry_sender = TelemetrySender(
        queues_dir=queues_dir, client_id=car_id, log_level=LOG_ERROR)

mission_planner = MissionPlanner(
    queues_dir, afcs_present=afcs_present, mission=mission)

sitl = SITL(
    queues_dir=queues_dir, position=home,
    car_id=car_id, post_telemetry=afcs_present, log_level=LOG_ERROR)


communication_gateway = CommunicationGateway(
    queues_dir=queues_dir, log_level=LOG_ERROR)
control_system = ControlSystem(queues_dir=queues_dir, log_level=LOG_INFO)

navigation_system = NavigationSystem(
    queues_dir=queues_dir, log_level=LOG_ERROR)

servos = Servos(queues_dir=queues_dir, log_level=LOG_ERROR)
cargo_bay = CargoBay(queues_dir=queues_dir, log_level=LOG_INFO)

# у нас получилось довольно много блоков, используем класс SystemComponentsContainer
# для упрощения рутинной работы с ними - таким образом мы собираем все блоки машинки в одном "кузове"
system_components = SystemComponentsContainer(
    components=[
        mission_sender,
        telemetry_sender,
        sitl,
        mission_planner,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system
    ] if afcs_present else [
        sitl,
        mission_planner,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system
    ])

system_components.start()

# ограничение поездки по времени
# параметр sleep - время в секундах,
# настройте этот параметр так, чтобы ваша машинка завершила маршрут
# в случае превышения времени выполнения ячейки на более чем 10 секунд от заданного, 
# допустимо перезапустить вычислительное ядро и повторно выполнить весь блокнот, штрафные очки за это не начисляются
# при условии, что повторный запуск закончился успешно
sleep(300)

# останавливаем все компоненты
system_components.stop()

# удалим все созданные компоненты
system_components.clean()

[ИНФО][QUEUES] создан каталог очередей
[ИНФО][QUEUES] регистрируем очередь planner.mqtt
[ИНФО][QUEUES] регистрируем очередь sitl.mqtt
[ИНФО][QUEUES] регистрируем очередь planner
[ИНФО][MISSION PLANNER] создана система планирования заданий
[ИНФО][QUEUES] регистрируем очередь sitl
[ИНФО][QUEUES] регистрируем очередь communication
[ИНФО][QUEUES] регистрируем очередь control
[ИНФО][CONTROL] создана система управления
[ИНФО][QUEUES] регистрируем очередь navigation
[ИНФО][QUEUES] регистрируем очередь servos
[ИНФО][QUEUES] регистрируем очередь cargo
[ИНФО][CARGO] создан компонент грузового отсека, отсек заблокирован
[ИНФО][MISSION PLANNER] старт системы планирования заданий
[ИНФО][CARGO] старт блока грузового отсека
[ИНФО][CONTROL] старт системы управления
[ИНФО][MISSION PLANNER] запрошена новая задача, отправляем получателям
[ИНФО][MISSION PLANNER] новая задача отправлена в коммуникационный шлюз
[ИНФО][CONTROL] установлена новая задача, начинаем следовать по маршруту, текущее время 15:23:13.

Process CargoBay-306:
Process ControlSystem-303:
Process MissionPlanner-300:


KeyboardInterrupt: 

Process TelemetrySender-299:
Process MissionSender-298:
Process NavigationSystem-304:
Process SITL-301:
Process Servos-305:
Process CommunicationGateway-302:
Traceback (most recent call last):


Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/user/Рабочий стол/cyberimmune-autonomy-chvt/src/cargo_bay.py", line 110, in run
    sleep(self._recalc_interval_sec)
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home

На этом модуль 2 завершён, надеемся, у вас всё получилось! 
На что можно ещё обратить вниманием - можно ли как-то улучшить маршрут для ускорения доставки? (без нарушения ПДД) - например, использовать более скоростные трассы или срезать какие-то участки.

Если идей быстрых улучшений нет, то пришло время перейти к самому интересному модулю и познакомиться с киберпрепятствиями!

Для работы над модулем 3 задания откройте блокнот cyberimmunity--autonomous-car-m3.